In [1]:
import os
from torchvision import transforms
from Dataset import Dataset
from data_loading import open_preprocess_photos, open_preprocess_photos_flip
from typing import List

# Model

In [2]:
from torchvision.models import resnet34, ResNet34_Weights, maxvit_t, MaxVit_T_Weights
import torch

resnet = resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)
#maxvit = maxvit_t(weights=MaxVit_T_Weights.IMAGENET1K_V1)

In [3]:
#maxvit.classifier = torch.nn.Identity()
resnet.fc = torch.nn.Identity()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Data

In [4]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

In [5]:
normal_dir: str = r'../chest_xray/train/NORMAL'
pneumo_dir: str = r'../chest_xray/train/PNEUMONIA'

assert os.path.exists(normal_dir) and os.path.isdir(normal_dir), "Normal dir isn't found or isn't a directory"
assert os.path.exists(pneumo_dir) and os.path.isdir(pneumo_dir), "Pneumonia dir isn't found or isn't a directory"

normal = open_preprocess_photos_flip(normal_dir, transform, (224, 224))
pneumonia = open_preprocess_photos(pneumo_dir, transform, (224, 224))

dataset_train = Dataset(normal, pneumonia[:len(normal)], 0, 1, 64)

In [5]:
normal_dir_test: str = r'../chest_xray/test/NORMAL'
pneumo_dir_test: str = r'../chest_xray/test/PNEUMONIA'

assert os.path.exists(normal_dir_test) and os.path.isdir(normal_dir_test), "Normal dir isn't found or isn't a directory"
assert os.path.exists(pneumo_dir_test) and os.path.isdir(pneumo_dir_test), "Pneumonia dir isn't found or isn't a directory"

normal_test = open_preprocess_photos_flip(normal_dir_test, transform, (224, 224))
pneumonia_test = open_preprocess_photos(pneumo_dir_test, transform, (224, 224))

dataset_test = Dataset(normal_test, pneumonia_test[:len(normal_test)], 0, 1, 64)

In [7]:
resnet = resnet.to(device)
#maxvit = maxvit.to(device)

In [9]:
resnet(dataset_test[0][0].to(device)).shape

torch.Size([64, 512])